In [15]:
import numpy as np
import pandas as pd
import pymongo

In [5]:
# 连接MongoDB
client = pymongo.MongoClient(host='localhost',port=27017)
# client = MongoClient('mongodb://localhost:27017/')

In [6]:
# 指定数据库
db = client.test
# db = client['test']

In [7]:
#  指定集合
collection = db.students
# collection = db['students']

In [8]:
# 插入数据
student = {'id':'20170101',
           'name':'jordan',
           'age':20,
           'gender':'male'}
result = collection.insert_one(student)
print(result)

In [9]:
print(result.inserted_id)

5cad5c91fb1c6e1028c65e90


In [11]:
# 
student1 = {'id':'20170102',
           'name':'tom',
           'age':21,
           'gender':'male'}
student2 = {'id':'20170101',
           'name':'mary',
           'age':22,
           'gender':'female'}
results = collection.insert_many([student1,student2])
print(results)
print(results.inserted_ids)

[ObjectId('5cad5d61fb1c6e1028c65e93'), ObjectId('5cad5d61fb1c6e1028c65e94')]


In [12]:
# 查询
result = collection.find_one({'name':'mary'})
print(type(result))
print(result)

<class 'dict'>
{'_id': ObjectId('5cad5d40fb1c6e1028c65e92'), 'id': '20170101', 'name': 'mary', 'age': 22, 'gender': 'female'}


In [30]:
result = collection.find({'gender':'male'})
print(type(result))
print(result)

<class 'pymongo.cursor.Cursor'>


In [31]:
for row in result:
    print(row)

{'_id': ObjectId('5cad5c91fb1c6e1028c65e90'), 'id': '20170101', 'name': 'jordan', 'age': 20, 'gender': 'male'}
{'_id': ObjectId('5cad5d40fb1c6e1028c65e91'), 'id': '20170102', 'name': 'tom', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('5cad5d61fb1c6e1028c65e93'), 'id': '20170102', 'name': 'tom', 'age': 21, 'gender': 'male'}


In [32]:
student3 = {'id':'20170103',
           'name':'jack',
           'age':20,
           'gender':'male'}
student4 = {'id':'20170104',
           'name':'hill',
           'age':23,
           'gender':'female'}
collection.insert_many([student3,student4])

In [33]:
results = collection.find({'age':{'$gt':20}})
for row in results:
    print(row)

{'_id': ObjectId('5cad5d40fb1c6e1028c65e91'), 'id': '20170102', 'name': 'tom', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('5cad5d40fb1c6e1028c65e92'), 'id': '20170101', 'name': 'mary', 'age': 22, 'gender': 'female'}
{'_id': ObjectId('5cad5d61fb1c6e1028c65e93'), 'id': '20170102', 'name': 'tom', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('5cad5d61fb1c6e1028c65e94'), 'id': '20170101', 'name': 'mary', 'age': 22, 'gender': 'female'}
{'_id': ObjectId('5cadba6cfb1c6e1028c65e96'), 'id': '20170104', 'name': 'hill', 'age': 23, 'gender': 'female'}


In [36]:
# 正则表达式
results = collection.find({'name':{'$regex':'^j'}})
for row in results:
    print(row)

{'_id': ObjectId('5cad5c91fb1c6e1028c65e90'), 'id': '20170101', 'name': 'jordan', 'age': 20, 'gender': 'male'}
{'_id': ObjectId('5cadba6cfb1c6e1028c65e95'), 'id': '20170103', 'name': 'jack', 'age': 20, 'gender': 'male'}


In [38]:
# 计数
results = collection.find({'age':20}).count()
print(results)

2


/Users/brianlee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [39]:
# 排序
results = collection.find().sort('name',pymongo.ASCENDING)
print([result['name'] for result in results])

['hill', 'jack', 'jordan', 'mary', 'mary', 'tom', 'tom']


In [40]:
# 偏移
results = collection.find().sort('name',pymongo.ASCENDING).skip(2)
print([result['name'] for result in results])

['jordan', 'mary', 'mary', 'tom', 'tom']


In [41]:
results = collection.find().sort('name',pymongo.ASCENDING).limit(2)
print([result['name'] for result in results])

['hill', 'jack']


In [42]:
from bson.objectid import ObjectId
collection.find({'_id':{"$gt":ObjectId('593278c815c2602678bb2b8d')}})

In [43]:
# 更新
condition = {'name':'tom'}
student = collection.find_one(condition)
student['name'] = 'air'
result = collection.update_one(condition,{'$set':student})
print(result.matched_count,result.modified_count)

1 1


In [46]:
condition = {'name':'mary'}
student = collection.find_one(condition)
student['name'] = 'alice'
result = collection.update_one(condition,{'$set':student})
print(result.matched_count,result.modified_count)

1 1


In [47]:
condition = {'age':20}
students = collection.find(condition)
result = collection.update_many(condition,{'$inc':{'age':1}})
print(result.matched_count,result.modified_count)

2 2


In [49]:
# 删除
collection.remove({'name':'jack'})

/Users/brianlee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  


{'n': 1, 'ok': 1.0}

In [50]:
# delete_one()即删除第一条符合条件的数据，delete_many()即删除所有符合条件的数据。
result1 = collection.delete_one({'name':'tom'})
print(result1.deleted_count)
result2 = collection.delete_many({'age':{'$gte':22}})
print(result2.deleted_count)

1
3


In [ ]:
# 聚合管道
collection.aggregate(
                     [
                       { '$match': { status: "A" } },
                       { '$group': { _id: "$cust_id", total: { $sum: "$amount" } } },
                       {' $sort': { total: -1 } },
                       { '$limit': 2 }
                     ])

https://docs.mongodb.com/manual/reference/operator/query/

#其他操作
PyMongo还提供了一些组合方法，如find_one_and_delete()、find_one_and_replace()和find_one_and_update()，
它们是查找后删除、替换和更新操作，其用法与上述方法基本一致。
另外，还可以对索引进行操作，相关方法有create_index()、create_indexes()和drop_index()等。

In [ ]:
# and
#collection.find( { '$and': [ { 'price': { '$ne': 1.99 } }, { 'price': { '$exists': true } } ] } )
# nor
# collection.find( { '$nor': [ { 'price': 1.99 }, { 'qty': { '$lt': 20 } }, { 'sale': true } ] } )
# not
#collection.find( { item: { $not: { $regex: "^p.*" } } } )
# or
# collection.find( { $or: [ { quantity: { $lt: 20 } }, { price: 10 } ] } )